In [1]:
import numpy as np
import os
import cv2 as cv2
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop


/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
Dir = 'asl'
catg = ["0","1","2","3","4","5","6","7","8","9","10","11","12","13",
"14","15","16","17","18","19","20","21","22","23","24","25","26","27",
"28","29","30","31","32","33","34","35"]

In [3]:
train = []
size = 200
def create_tr():
    for c in catg:
        path=os.path.join(Dir, c)
        class_nnn= catg.index(c)
        for img in os.listdir(path):
            imgax = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE) #grayscale
            imgar = cv2.resize(imgax, (size, size))
            train.append([imgar, class_nnn])


create_tr()

In [4]:
print("Number of elements in training set: ", len(train))
random.shuffle(train)

tX = []
tY = []

for features, label in train:
    tX.append(features)
    tY.append(label)
    
tX = np.array(tX).reshape(-1, size, size, 1) #1 not 3
tX = tX / 255.0

Number of elements in training set:  25291


In [8]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.99):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks1 = myCallback()
checkpoint_path = "Pretrained_Model/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [9]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, kernel_size = [3,3], padding = 'same', activation = 'relu', input_shape = tX.shape[1:]),
        tf.keras.layers.Conv2D(64, kernel_size = [3,3], padding = 'same', activation = 'relu'),
        tf.keras.layers.MaxPool2D(pool_size = [3,3]),
        tf.keras.layers.Conv2D(128, kernel_size = [3,3], padding = 'same', activation = 'relu'),
        tf.keras.layers.MaxPooling2D(3,3),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2), 
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'), 
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dense(1024, activation = 'relu'),
        tf.keras.layers.Dense(512, activation = 'relu'),
        tf.keras.layers.Flatten(), 
        tf.keras.layers.Dense(1013, activation='relu'), 
        tf.keras.layers.Dense(36, activation='softmax')  
    ])



model.summary()
model.compile(optimizer=RMSprop(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics = ['acc'])

#model.fit(tX, tY, batch_size=25, validation_split=0.10, epochs = 10, callbacks=[callbacks1, cp_callback])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 200, 200, 64)      640       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 200, 200, 64)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 66, 66, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 66, 66, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 22, 22, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 20, 20, 64)        73792     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 10, 10, 64)       

In [10]:

model.load_weights(checkpoint_path)
loss,acc = model.evaluate(tX, tY)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


25291/25291 [==============================] - 1183s 47ms/sample - loss: 0.0290 - acc: 0.9898
Restored model, accuracy: 98.98%


In [12]:
zzx = ""
while zzx != "exit":

      path_to_pic = input("Enter the path to the image you would like to predict: ")
      imgax4 = cv2.imread(os.path.join(path_to_pic), cv2.IMREAD_GRAYSCALE)
      imgar4 = cv2.resize(imgax4, (size, size))

      ttX = []
      for feature in imgar4:
          ttX.append(feature)
          
      ttX = np.array(ttX).reshape(-1, size, size, 1)
      eer = model.predict(ttX)
      z = np.where(eer == 1)
      if z[1]>9:
          prediction = chr((z[1]-10) + 65)
          print(prediction)
      else:
          prediction = int(z[1])
          print(prediction)
      zzx = input("Enter exit to stop or return to keep predicting.")

Enter the path to the image you would like to predict: /Users/rajdeepbandopadhyay/Desktop/SignLanguage_ML/photo.jpg
R
Enter exit to stop or return to keep predicting./Users/rajdeepbandopadhyay/Desktop/SignLanguage_ML/photo.jpg
Enter the path to the image you would like to predict: /Users/rajdeepbandopadhyay/Desktop/SignLanguage_ML/photo.jpg
R
Enter exit to stop or return to keep predicting.e
Enter the path to the image you would like to predict: /Users/rajdeepbandopadhyay/Desktop/rr.jpg
N
Enter exit to stop or return to keep predicting./Users/rajdeepbandopadhyay/Desktop/p.jpg
Enter the path to the image you would like to predict: /Users/rajdeepbandopadhyay/Desktop/p.jpg
O
Enter exit to stop or return to keep predicting.exit
